In [10]:
# !pip install -q google-generativeai pandas

import os, json, time, pathlib
import pandas as pd
import google.generativeai as genai

In [ ]:
###################################################
GEMINI_API_KEY = "REMOVED FOR SECURITY REASONS"
###################################################

MODEL_NAME = "gemini-2.5-flash"  
SUBSET_JSONL = "data/medquestions_subset_500.jsonl"

RESULTS_DIR = pathlib.Path("results/gemini")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_JSONL = RESULTS_DIR / "results_gemini_medquestions150.jsonl"
RESULTS_CSV   = RESULTS_DIR / "results_gemini_medquestions150.csv"

PACING_SLEEP = 10.0   
RETRY_BACKOFFS = [1, 2] 

print("Results folder:", RESULTS_DIR)


Results folder: results\gemini


In [12]:
# Run constants
SLICE = (0, 150)   
DRY_RUN = False        
PACING_SLEEP = 10.0    
RETRY_BACKOFFS = [1, 2] 

In [13]:
# Helpers
def append_jsonl(path, record):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def load_done_ids(path):
    done = set()
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    obj = json.loads(line)
                    qid = obj.get("qid")
                    if qid:
                        done.add(qid)
                except Exception:
                    pass
    return done

In [14]:
# Gemini Q&A function
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(MODEL_NAME)

def ask_gemini(question: str) -> str:
    prompt = (
        "You are a concise medical Q&A assistant. "
        "Answer briefly and directly based on general medical knowledge.\n\n"
        f"Question: {question}"
    )
    resp = model.generate_content(prompt)
    return getattr(resp, "text", str(resp)).strip()

In [15]:
subset_all = [json.loads(l) for l in open(SUBSET_JSONL, "r", encoding="utf-8")]

start_idx, end_idx = SLICE
assert 0 <= start_idx < end_idx <= 500, "Slice must be within 0..500"
subset = subset_all[start_idx:end_idx]

if DRY_RUN:
    subset = subset[:3]

done = load_done_ids(RESULTS_JSONL)
subset = [rec for rec in subset if rec["qid"] not in done]

print(f"Slice: {SLICE}, dry_run={DRY_RUN}")
print(f"Planned in slice: {end_idx - start_idx}")
print(f"Already completed (skipped): {len(done)}")
print(f"Will ask now: {len(subset)}")

# For GPT to reuse the exact same 150
qid_list_path = pathlib.Path("data") / f"run_qids_{start_idx}_{end_idx}.txt"
qid_list_path.parent.mkdir(parents=True, exist_ok=True)
with open(qid_list_path, "w", encoding="utf-8") as f:
    for rec in subset_all[start_idx:end_idx]:
        f.write(rec["qid"] + "\n")
print("Saved QID list:", qid_list_path)

Slice: (0, 150), dry_run=False
Planned in slice: 150
Already completed (skipped): 0
Will ask now: 150
Saved QID list: data\run_qids_0_150.txt


In [19]:
import json, pathlib

subset_all = [json.loads(l) for l in open(SUBSET_JSONL, "r", encoding="utf-8")]

start_idx, end_idx = SLICE
assert 0 <= start_idx < end_idx <= 150, "Slice must be within 0..150"
slice_recs = subset_all[start_idx:end_idx]

qid_list_path = pathlib.Path("data") / f"run_qids_{start_idx}_{end_idx}.txt"
qid_list_path.parent.mkdir(parents=True, exist_ok=True)
with open(qid_list_path, "w", encoding="utf-8") as f:
    for r in slice_recs:
        f.write(r["qid"] + "\n")

# Create questions only JSONL (NO answers)
QUESTIONS_ONLY_JSONL = pathlib.Path("data") / f"questions_only_{start_idx}_{end_idx}.jsonl"
with open(QUESTIONS_ONLY_JSONL, "w", encoding="utf-8") as f:
    for r in slice_recs:
        f.write(json.dumps({"qid": r["qid"], "question": r["question"]}, ensure_ascii=False) + "\n")

print("Slice:", SLICE, "dry_run:", DRY_RUN)
print("Saved QID list:", qid_list_path)
print("Saved questions-only JSONL:", QUESTIONS_ONLY_JSONL)

Slice: (0, 150) dry_run: False
Saved QID list: data\run_qids_0_150.txt
Saved questions-only JSONL: data\questions_only_0_150.jsonl


In [21]:
import json, time

subset_q = [json.loads(l) for l in open(QUESTIONS_ONLY_JSONL, "r", encoding="utf-8")]
done = load_done_ids(RESULTS_JSONL) 

subset_q = [rec for rec in subset_q if rec["qid"] not in done]

print(f"Will ask now: {len(subset_q)}")

count, errors = 0, 0
t_run0 = time.time()

for rec in subset_q:
    qid = rec["qid"]
    question = rec["question"]

    pred = ""
    status = "ok"
    err = None
    t0 = time.time()

    for wait_s in [0] + RETRY_BACKOFFS:
        if wait_s:
            time.sleep(wait_s)
        try:
            pred = ask_gemini(question) 
            err = None
            status = "ok"
            break
        except Exception as e:
            err = str(e)
            status = "error"
    else:
        errors += 1
        pred = ""

    latency_ms = int((time.time() - t0) * 1000)

    append_jsonl(
        RESULTS_JSONL,
        {
            "qid": qid,
            "model": MODEL_NAME,
            "question": question,
            "model_answer": pred,
            "latency_ms": latency_ms,
            "status": status,
            "error": err, 
        },
    )

    count += 1
    if count % 10 == 0:
        elapsed = (time.time() - t_run0) / 60
        print(f"Progress: {count}/{SLICE[1] - SLICE[0]} (~{elapsed:.1f} min)")

    time.sleep(PACING_SLEEP)

elapsed_total = (time.time() - t_run0) / 60
print(f"Completed new entries: {count} | Errors: {errors} | Elapsed min: {elapsed_total:.1f}")


Will ask now: 150
Progress: 10/150 (~1.9 min)
Progress: 20/150 (~4.1 min)
Progress: 30/150 (~6.2 min)
Progress: 40/150 (~8.4 min)
Progress: 50/150 (~10.6 min)
Progress: 60/150 (~12.7 min)
Progress: 70/150 (~14.8 min)
Progress: 80/150 (~16.9 min)
Progress: 90/150 (~19.0 min)
Progress: 100/150 (~21.1 min)
Progress: 110/150 (~23.2 min)
Progress: 120/150 (~106.3 min)
Progress: 130/150 (~108.4 min)
Progress: 140/150 (~110.6 min)
Progress: 150/150 (~112.7 min)
Completed new entries: 150 | Errors: 0 | Elapsed min: 112.8


In [22]:
df = pd.read_json(RESULTS_JSONL, lines=True)
df.to_csv(RESULTS_CSV, index=False)

print("Saved:")
print(" ", RESULTS_JSONL)
print(" ", RESULTS_CSV)

print("\nSummary:")
print(" Slice size:", SLICE[1] - SLICE[0])
print(" Rows logged in file:", len(df))
print(" OK rows:", (df["status"] == "ok").sum())
print(" ERROR rows:", (df["status"] == "error").sum())
print(" Non-empty answers:", (df["model_answer"].str.len() > 0).sum())

df.head(3)

Saved:
  results\gemini\results_gemini_medquestions150.jsonl
  results\gemini\results_gemini_medquestions150.csv

Summary:
 Slice size: 150
 Rows logged in file: 150
 OK rows: 150
 ERROR rows: 0
 Non-empty answers: 150


,qid,model,question,model_answer,latency_ms,status,error
0,medquestions-500-0000,gemini-2.5-flash,What are the genetic changes related to famili...,Germline mutation in the *APC* gene.,818,ok,NaN
1,medquestions-500-0001,gemini-2.5-flash,What are the treatments for Noonan syndrome ?,Treatment for Noonan syndrome is supportive an...,1731,ok,NaN
2,medquestions-500-0002,gemini-2.5-flash,How to diagnose National Hormone and Pituitary...,NHPP is a program providing information for in...,2260,ok,NaN
